In [38]:
import pathlib
from lnas import LnasFormat
import numpy as np
import pandas as pd

lnas_path = "output/low_rise_building_Rabat_Sale.lnas"

lnas_fmt = LnasFormat.from_file(pathlib.Path(lnas_path))
lnas_geo = lnas_fmt.geometry

tri_normals = lnas_geo.normals

# Triangles that point below
z_down_triangles = tri_normals[:, 2] < -0.5
z_up_triangles = tri_normals[:, 2] > 0.5

tri_down = np.arange(len(z_down_triangles))[z_down_triangles]
tri_up = np.arange(len(z_up_triangles))[z_up_triangles]

lnas_filtered_down = lnas_fmt.filter_triangles(z_down_triangles)
lnas_filtered_up = lnas_fmt.filter_triangles(z_up_triangles)

lnas_fmt.geometry.export_stl(pathlib.Path("output/maroco_full.stl"))
lnas_filtered_down.geometry.export_stl(pathlib.Path("output/maroco_down.stl"))
lnas_filtered_up.geometry.export_stl(pathlib.Path("output/maroco_up.stl"))



In [30]:
from scipy.spatial import KDTree

geom_up = lnas_filtered_up.geometry
tri_up_centroids = geom_up.triangle_vertices.mean(axis=1)
# Remove z axis
tri_up_centroids = tri_up_centroids[:, :2]

kd = KDTree(tri_up_centroids)
tri_up_centroids.shape, geom_up.triangle_vertices.shape

((8205, 2), (8205, 3, 3))

In [31]:
geom_down = lnas_filtered_down.geometry
tri_down_centroids = geom_down.triangle_vertices.mean(axis=1)
# Remove z axis
tri_down_centroids = tri_down_centroids[:, :2]
tri_down_centroids.shape

(2711, 2)

In [32]:
dist, idx_cmp = kd.query(tri_down_centroids)
idx_cmp.shape, dist.shape

((2711,), (2711,))

In [42]:
tri_cmp_up = tri_up[idx_cmp]

df_cmp = pd.DataFrame({"point_idx": tri_down, "idx_cmp": tri_cmp_up})
df_cmp

,point_idx,idx_cmp
0,3,4258
1,4,817
2,5,2954
3,7,13510
4,13,14105
...,...,...
2706,14847,10547
2707,14848,12317
2708,14849,11950
2709,14858,12937


In [2]:
import pandas as pd
# The dataframe was treated to be only one store
df_path = "output/pressure_maroco.hdf"

df_use = pd.read_hdf(df_path)
print(df_use)

       point_idx       rho  time_step
0              0  0.995869     4000.0
1              1  0.995857     4000.0
2              2  0.995754     4000.0
3              3  0.989892     4000.0
4              4  0.989654     4000.0
...          ...       ...        ...
14863      14863  0.993919     6999.0
14864      14864  0.993493     6999.0
14865      14865  0.993850     6999.0
14866      14866  0.993504     6999.0
14867      14867  0.994859     6999.0

[44604000 rows x 3 columns]


In [54]:
tile_bool_down = np.tile(z_down_triangles, reps=len(df_use) // len(z_down_triangles))

df_down = df_use[tile_bool_down]

tile_bool_up = np.tile(z_up_triangles, reps=len(df_use) // len(z_up_triangles))

df_up = df_use[tile_bool_up]
df_up = df_up.rename(columns={"rho": "rho_up", "point_idx": "idx_cmp"})

df_up.head(), df_down.head()


(    idx_cmp    rho_up  time_step
 0         0  0.995869     4000.0
 1         1  0.995857     4000.0
 2         2  0.995754     4000.0
 6         6  0.995668     4000.0
 15       15  0.986070     4000.0,
     point_idx       rho  time_step
 3           3  0.989892     4000.0
 4           4  0.989654     4000.0
 5           5  0.988977     4000.0
 7           7  0.989853     4000.0
 13         13  0.986489     4000.0)

In [55]:
df_down.shape

(8133000, 3)

In [59]:
df_full = pd.merge(df_down, df_cmp, on="point_idx", how="left")
df_full

,point_idx,rho,time_step,idx_cmp
0,3,0.989892,4000.0,4258
1,4,0.989654,4000.0,817
2,5,0.988977,4000.0,2954
3,7,0.989853,4000.0,13510
4,13,0.986489,4000.0,14105
...,...,...,...,...
8132995,14847,0.995765,6999.0,10547
8132996,14848,0.994963,6999.0,12317
8132997,14849,0.994971,6999.0,11950
8132998,14858,0.994286,6999.0,12937


In [60]:
df_liquid = pd.merge(df_full, df_up, on=["idx_cmp", "time_step"], how="left")
df_liquid

,point_idx,rho,time_step,idx_cmp,rho_up
0,3,0.989892,4000.0,4258,0.985164
1,4,0.989654,4000.0,817,0.985340
2,5,0.988977,4000.0,2954,0.985278
3,7,0.989853,4000.0,13510,0.983206
4,13,0.986489,4000.0,14105,0.980847
...,...,...,...,...,...
8132995,14847,0.995765,6999.0,10547,0.991078
8132996,14848,0.994963,6999.0,12317,0.994523
8132997,14849,0.994971,6999.0,11950,0.994520
8132998,14858,0.994286,6999.0,12937,0.993904


In [63]:
df_liquid["rho_liq"] = df_liquid["rho_up"] - df_liquid["rho"]
df_liquid

,point_idx,rho,time_step,idx_cmp,rho_up,rho_liq
0,3,0.989892,4000.0,4258,0.985164,-0.004728
1,4,0.989654,4000.0,817,0.985340,-0.004313
2,5,0.988977,4000.0,2954,0.985278,-0.003700
3,7,0.989853,4000.0,13510,0.983206,-0.006647
4,13,0.986489,4000.0,14105,0.980847,-0.005642
...,...,...,...,...,...,...
8132995,14847,0.995765,6999.0,10547,0.991078,-0.004688
8132996,14848,0.994963,6999.0,12317,0.994523,-0.000440
8132997,14849,0.994971,6999.0,11950,0.994520,-0.000450
8132998,14858,0.994286,6999.0,12937,0.993904,-0.000382
